In [1]:
import os
import pandas as pd
from sqlalchemy import create_engine

# PostgreSQL connection details
user = "postgres"
password = "newpassword"
host = "localhost"
port = "5432"
database = "newspaper"

# Create SQLAlchemy engine
engine = create_engine(f"postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}")

# Path to your datasets folder
folder = r"C:\Users\brdrv\OneDrive\Desktop\NewPaper\datasets"

# Loop through all CSV files in the folder
for file in os.listdir(folder):
    if file.endswith(".csv"):
        file_path = os.path.join(folder, file)
        table_name = os.path.splitext(file)[0].lower()

        try:
            # Read CSV
            df = pd.read_csv(file_path)

            # Load into PostgreSQL
            df.to_sql(table_name, engine, if_exists="replace", index=False)

            print(f"✅ Loaded {file} into table '{table_name}'")

        except Exception as e:
            print(f"❌ Failed to load {file}: {e}")



✅ Loaded dim_ad_category.csv into table 'dim_ad_category'
✅ Loaded dim_city.csv into table 'dim_city'
✅ Loaded dim_date.csv into table 'dim_date'
✅ Loaded fact_ad_revenue.csv into table 'fact_ad_revenue'
✅ Loaded fact_city_readiness.csv into table 'fact_city_readiness'
✅ Loaded fact_digital_pilot.csv into table 'fact_digital_pilot'
✅ Loaded fact_print_sales.csv into table 'fact_print_sales'


In [ ]:
df=pd.read_sql("select * from fact_print_sales;",engine)


In [7]:
df.City_ID

0      C010
1      C010
2      C010
3      C010
4      C010
       ... 
715    C007
716    C007
717    C007
718    C007
719    C007
Name: City_ID, Length: 720, dtype: object

In [11]:
import pandas as pd

query = """
WITH monthly_sales AS (
    SELECT 
        c.city,
        TO_CHAR(f.date, 'YYYY-MM') AS month,
        SUM(f.net_circulation) AS net_circulation
    FROM fact_print_sales f
    JOIN dim_city c ON c.city_id = f.City_ID
    WHERE f.year BETWEEN 2019 AND 2024
    GROUP BY c.city, TO_CHAR(f.date, 'YYYY-MM')
),
with_lag AS (
    SELECT 
        city,
        month,
        net_circulation,
        LAG(net_circulation) OVER (PARTITION BY city ORDER BY month) AS prev_circulation
    FROM monthly_sales
)
SELECT 
    city,
    month,
    net_circulation,
    (prev_circulation - net_circulation) AS decline
FROM with_lag
WHERE prev_circulation IS NOT NULL
ORDER BY decline DESC
LIMIT 3;
"""

df = pd.read_sql(query, engine)
print(df)




ProgrammingError: (psycopg2.errors.UndefinedColumn) column f.city_id does not exist
LINE 8:     JOIN dim_city c ON c.city_id = f.City_ID
                                           ^
HINT:  Perhaps you meant to reference the column "c.city_id".

[SQL: 
WITH monthly_sales AS (
    SELECT 
        c.city,
        TO_CHAR(f.date, 'YYYY-MM') AS month,
        SUM(f.net_circulation) AS net_circulation
    FROM fact_print_sales f
    JOIN dim_city c ON c.city_id = f.City_ID
    WHERE f.year BETWEEN 2019 AND 2024
    GROUP BY c.city, TO_CHAR(f.date, 'YYYY-MM')
),
with_lag AS (
    SELECT 
        city,
        month,
        net_circulation,
        LAG(net_circulation) OVER (PARTITION BY city ORDER BY month) AS prev_circulation
    FROM monthly_sales
)
SELECT 
    city,
    month,
    net_circulation,
    (prev_circulation - net_circulation) AS decline
FROM with_lag
WHERE prev_circulation IS NOT NULL
ORDER BY decline DESC
LIMIT 3;
]
(Background on this error at: https://sqlalche.me/e/20/f405)